<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [4]:

# 2. 런타임 확인용 (설치 잘 되었는지)
import torch
import timm
print("Torch version:", torch.__version__)
print("Timm model test:", timm.list_models("efficientnet*"))

Torch version: 2.6.0+cu124
Timm model test: ['efficientnet_b0', 'efficientnet_b0_g8_gn', 'efficientnet_b0_g16_evos', 'efficientnet_b0_gn', 'efficientnet_b1', 'efficientnet_b1_pruned', 'efficientnet_b2', 'efficientnet_b2_pruned', 'efficientnet_b3', 'efficientnet_b3_g8_gn', 'efficientnet_b3_gn', 'efficientnet_b3_pruned', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_b8', 'efficientnet_blur_b0', 'efficientnet_cc_b0_4e', 'efficientnet_cc_b0_8e', 'efficientnet_cc_b1_8e', 'efficientnet_el', 'efficientnet_el_pruned', 'efficientnet_em', 'efficientnet_es', 'efficientnet_es_pruned', 'efficientnet_h_b5', 'efficientnet_l2', 'efficientnet_lite0', 'efficientnet_lite1', 'efficientnet_lite2', 'efficientnet_lite3', 'efficientnet_lite4', 'efficientnet_x_b3', 'efficientnet_x_b5', 'efficientnetv2_l', 'efficientnetv2_m', 'efficientnetv2_rw_m', 'efficientnetv2_rw_s', 'efficientnetv2_rw_t', 'efficientnetv2_s', 'efficientnetv2_xl']


In [ ]:
# 전략 1. 데이터 전처리
#       - 이미지 사이즈 및 중심으로 정렬
#       - 밝기/대비 조정
#       - 이미지 노이즈 제거
# 전략 2. 데이터 증강
#       - 기본 Augmentation 차량의 각도나 배경 고려
#       - Mixup/cutmix (클래스 간 경계를 부드럽게 학습)

In [8]:
# 학습, Validation Accuracy 측정, EarlyStopping, 모델 저장 포함
import os, random
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import timm

# ========================
# 설정
SEED = 42
BATCH_SIZE = 32
EPOCHS = 20
FOLDS = 5
TTA_ROUNDS = 2
IMG_SIZE = 224
PATIENCE = 3  # 조기종료 기준
os.makedirs("models", exist_ok=True)
os.makedirs("val_logs", exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)

# ========================
# 클래스 통합
merge_map = {
    'K5_3세대_하이브리드_2020_2022': 'K5_하이브리드_3세대_2020_2023',
    '디_올뉴니로_2022_2025': '디_올_뉴_니로_2022_2025',
    '718_박스터_2017_2024': '박스터_718_2017_2024',
}
ROOT_DIR = '/content/drive/MyDrive/open'
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
TEST_DIR = os.path.join(ROOT_DIR, 'test')
TEST_CSV = os.path.join(ROOT_DIR, 'test.csv')
SUBMIT_CSV = os.path.join(ROOT_DIR, 'sample_submission.csv')

raw_classes = os.listdir(TRAIN_DIR)
merged_classes = sorted(set([merge_map.get(c, c) for c in raw_classes]))
class_to_idx = {c: i for i, c in enumerate(merged_classes)}
idx_to_class = {i: c for c, i in class_to_idx.items()}

# ========================
# Dataset 정의
class CarDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.labels is not None:
            return img, self.labels[idx]
        else:
            return img

# ========================
# Transform
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
# ========================
# 데이터 로딩
train_image_paths, train_labels = [], []
for folder in raw_classes:
    unified = merge_map.get(folder, folder)
    for f in os.listdir(os.path.join(TRAIN_DIR, folder)):
        train_image_paths.append(os.path.join(TRAIN_DIR, folder, f))
        train_labels.append(class_to_idx[unified])

test_df = pd.read_csv(TEST_CSV)
test_image_paths = [os.path.join(TEST_DIR, p) for p in test_df['img_path']]
submission = pd.read_csv(SUBMIT_CSV)
preds_396 = np.zeros((len(test_df), 396))
val_acc_log = []

# ========================
# KFold 학습
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_image_paths, train_labels)):
    print(f"\n🌀 Fold {fold+1}/{FOLDS}")
    X_train = [train_image_paths[i] for i in train_idx]
    y_train = [train_labels[i] for i in train_idx]
    X_val = [train_image_paths[i] for i in val_idx]
    y_val = [train_labels[i] for i in val_idx]

    train_ds = CarDataset(X_train, y_train, transform=train_transform)
    val_ds = CarDataset(X_val, y_val, transform=test_transform)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = timm.create_model('efficientnet_b5', pretrained=True, num_classes=len(merged_classes)).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    best_acc, patience_counter = 0, 0
    for epoch in range(EPOCHS):
        model.train()
        for images, labels in tqdm(train_loader, desc=f"[Fold {fold+1}][Epoch {epoch+1}]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # validation accuracy
        model.eval()
        preds, true = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                outputs = model(images)
                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                true.extend(labels.numpy())
        acc = accuracy_score(true, preds)
        print(f"📌 [Fold {fold+1}][Epoch {epoch+1}] Val Accuracy: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), f"models/best_fold{fold+1}.pth")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("⏹️ Early stopping triggered.")
                break

    val_acc_log.append(best_acc)

    # Confusion Matrix 저장
    cm = confusion_matrix(true, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, cmap="Blues", xticklabels=False, yticklabels=False)
    plt.title(f"Confusion Matrix Fold {fold+1}")
    plt.savefig(f"val_logs/confmat_fold{fold+1}.png")
    plt.close()

    # Inference with TTA
    model.load_state_dict(torch.load(f"models/best_fold{fold+1}.pth"))
    model.eval()
    fold_preds = np.zeros((len(test_df), len(merged_classes)))
    for _ in range(TTA_ROUNDS):
        test_ds = CarDataset(test_image_paths, transform=test_transform)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
        tta_preds = []
        with torch.no_grad():
            for batch in test_loader:
                imgs = batch.to(device)
                probs = torch.softmax(model(imgs), dim=1).cpu().numpy()
                tta_preds.append(probs)
        fold_preds += np.vstack(tta_preds) / TTA_ROUNDS

    # 통합 → 396 클래스 맵핑
    for col in submission.columns[1:]:
        mapped_col = merge_map.get(col, col)
        if mapped_col in class_to_idx:
            preds_396[:, submission.columns.get_loc(col)-1] += fold_preds[:, class_to_idx[mapped_col]] / FOLDS

# ========================
# 최종 제출 파일 저장
submission.iloc[:, 1:] = preds_396
submission.to_csv("submission_final.csv", index=False)
print("✅ 최종 제출 파일 저장 완료: submission_final.csv")
print(f"📊 Fold별 Val Accuracy: {val_acc_log}")
print(f"📈 평균 Validation Accuracy: {np.mean(val_acc_log):.4f}")



🌀 Fold 1/5


[Fold 1][Epoch 1]:   0%|          | 2/829 [00:51<5:57:21, 25.93s/it]